Imports

In [95]:
import numpy as np
np.set_printoptions(threshold=10000,suppress=True)
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

## Données
Charger les données

In [96]:
def load_data(path):
    data = pd.read_csv('data/' + path, header=None, sep='\t')
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    print("X shape:", X.shape)
    print("y shape:", y.shape)
    return X, y

X, y = load_data('iris.txt')

# afficher la premiere ligne de X et y
print(X[:1])
print(y[:1])



X shape: (150, 4)
y shape: (150,)
[[5.1 3.5 1.4 0.2]]
[1]


Découpage de la base d'apprentissage

In [97]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42, stratify=y)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (100, 4)
X_test shape: (50, 4)


## Perceptron Multi-classe

In [98]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

perceptron = Perceptron(max_iter=1000, eta0=0.1, random_state=42)
perceptron.fit(X_train, y_train)

y_pred = perceptron.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.2f}")


Accuracy: 0.80


In [99]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

def display_metrics(y_test, y_pred):
    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    print("Matrice de confusion:")
    print(cm)

    # Accuracy globale
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nAccuracy globale: {accuracy:.4f}")

    # Précision pour chaque classe
    precision = precision_score(y_test, y_pred, average=None, zero_division=0)
    print("\nPrécision par classe:")
    for i, p in enumerate(precision):
        print(f"  Classe {i+1}: {p:.4f}")

    # Rappel pour chaque classe
    recall = recall_score(y_test, y_pred, average=None, zero_division=0)
    print("\nRappel par classe:")
    for i, r in enumerate(recall):
        print(f"  Classe {i+1}: {r:.4f}")

display_metrics(y_test, y_pred)

Matrice de confusion:
[[15  1  0]
 [ 0  8  9]
 [ 0  0 17]]

Accuracy globale: 0.8000

Précision par classe:
  Classe 1: 1.0000
  Classe 2: 0.8889
  Classe 3: 0.6538

Rappel par classe:
  Classe 1: 0.9375
  Classe 2: 0.4706
  Classe 3: 1.0000


## Perceptron Multi Couche

In [100]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(3,), max_iter=1000, random_state=42,  alpha=0.001)

mlp.fit(X_train, y_train)

y_pred_mlp = mlp.predict(X_test)

display_metrics(y_test, y_pred_mlp)

Matrice de confusion:
[[16  0  0]
 [ 0 16  1]
 [ 0  0 17]]

Accuracy globale: 0.9800

Précision par classe:
  Classe 1: 1.0000
  Classe 2: 1.0000
  Classe 3: 0.9444

Rappel par classe:
  Classe 1: 1.0000
  Classe 2: 0.9412
  Classe 3: 1.0000


Version normalisée

In [101]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp.fit(X_train_scaled, y_train)

y_pred_mlp_scaled = mlp.predict(X_test_scaled)

display_metrics(y_test, y_pred_mlp_scaled)

Matrice de confusion:
[[16  0  0]
 [ 1 15  1]
 [ 0  3 14]]

Accuracy globale: 0.9000

Précision par classe:
  Classe 1: 0.9412
  Classe 2: 0.8333
  Classe 3: 0.9333

Rappel par classe:
  Classe 1: 1.0000
  Classe 2: 0.8824
  Classe 3: 0.8235


## Application à tous les datasets

In [104]:
def pipeline(path, seed=42):
    X, y = load_data(path)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=seed, stratify=y)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Multi classe
    perceptron = Perceptron(max_iter=1000, eta0=0.1, random_state=seed)
    perceptron.fit(X_train_scaled, y_train)
    y_pred = perceptron.predict(X_test_scaled)
    print("\nPerceptron:")
    display_metrics(y_test, y_pred)
    
    # Multi couche
    mlp = MLPClassifier(hidden_layer_sizes=(3,), max_iter=1000, random_state=seed,  alpha=0.001)
    mlp.fit(X_train_scaled, y_train)
    y_pred = mlp.predict(X_test_scaled)
    print("\nPerceptron Multi-Couche:")  
    display_metrics(y_test, y_pred)

pipeline('iris.txt')

X shape: (150, 4)
y shape: (150,)

Perceptron:
Matrice de confusion:
[[15  1  0]
 [ 5  7  5]
 [ 0  2 15]]

Accuracy globale: 0.7400

Précision par classe:
  Classe 1: 0.7500
  Classe 2: 0.7000
  Classe 3: 0.7500

Rappel par classe:
  Classe 1: 0.9375
  Classe 2: 0.4118
  Classe 3: 0.8824

Perceptron Multi-Couche:
Matrice de confusion:
[[16  0  0]
 [ 1 15  1]
 [ 0  3 14]]

Accuracy globale: 0.9000

Précision par classe:
  Classe 1: 0.9412
  Classe 2: 0.8333
  Classe 3: 0.9333

Rappel par classe:
  Classe 1: 1.0000
  Classe 2: 0.8824
  Classe 3: 0.8235
